In [1]:
# Import Modules 

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
# When importing nltk for the first time, we need to also download a few built-in libraries
!pip install nltk
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords 
from nltk import word_tokenize

stop_words = set(stopwords.words('english')) 


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zacharie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/zacharie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zacharie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/zacharie/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
%load_ext autoreload
%autoreload 2

data = pd.read_csv("raw_data/Recipes/Food Ingredients and Recipe Dataset with Image Name Mapping.csv")

In [3]:
l = ['cup',
 'tablespoon',
 'teaspoon',
 'chopped',
 'salt',
 'fresh',
 'ounce',
 'large',
 'ground',
 'sugar',
 'pepper',
 'pound',
 'olive',
 'finely',
 'inch',
 'cut',
 'kosher',
 'sliced',
 'butter',
 'plus',
 'garlic',
 'leaf',
 'black',
 'divided',
 'unsalted',
 'freshly',
 'red',
 'peeled',
 'white',
 'flour',
 'thinly',
 'piece',
 'whole',
 'medium',
 'small',
 'extra',
 'water',
 'stick',
 'purpose',
 'virgin',
 'dried',
 'tbsp',
 'seed',
 'tsp',
 'green',
 'halved',
 'oz',
 'minced',
 'packed',
 'baking',
 'g',
 'thick',
 'slice',
 'parsley',
 'serving',
 'lb',
 'optional',
 'dry',
 'extract',
 'brown',
 'temperature',
 'room',
 'trimmed',
 'heavy',
 'crushed',
 'sprig',
 'lengthwise',
 'cilantro',
 'removed',
 'preferably',
 'diced',
 'drained',
 'bunch',
 'half']

stop_words.update(l)

In [4]:
import string

def text_to_list(text):
    text = text.replace("['", "")
    text = text.replace("']", "")
    my_list = text.split("', '")
    return my_list

def remove_punctuation(text):
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ') 
    return text

def remove_nonalpha(text):
    list_of_words = text.split(' ')
    list_clean = []
    for words in list_of_words:
        words_only = ''.join([i for i in words if i.isalpha()])
        list_clean.append(words_only)
    final_words = ' '.join(list_clean)
    return final_words

def lowercase(text): 
    lowercased = text.lower() 
    return lowercased

def remove_stopwords(text):
    tokenized = word_tokenize(text)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

from nltk.stem import WordNetLemmatizer

def lemma(text):
    lemmatizer = WordNetLemmatizer() # Initiate lemmatizer
    lemmatized = [lemmatizer.lemmatize(word) for word in text] # Lemmatize
    lemmatized_string = " ".join(lemmatized)
    return lemmatized_string

def pun_list(liste):
    my_list = [remove_punctuation(text) for text in liste]
    return my_list
def num_list(liste):
    my_list = [remove_nonalpha(text) for text in liste]
    return my_list
def low_list(liste):
    my_list = [lowercase(text) for text in liste]
    return my_list
def sto_list(liste):
    my_list = [remove_stopwords(text) for text in liste]
    return my_list
def lem_list(liste):
    my_list = [lemma(text) for text in liste]
    return my_list
    

In [5]:




default_ingredients = ['apple', 'banana', 'beef', 'blueberries',
                       'bread', 'butter', 'carrot', 'cheese',
                       'chicken', 'chicken breast', 'chocolate',
                       'corn', 'eggs', 'flour', 'goat cheese',
                       'green beans', 'ground beef', 'ham', 'heavy cream',
                       'lime', 'milk', 'mushrooms','onion',
                       'potato', 'shrimp', 'spinach', 'strawberries',
                       'sugar', 'sweet potato', 'tomato']

l_ingredients = sto_list(default_ingredients)
l_ingredients = lem_list(l_ingredients)
l_ingredients = [word for word in l_ingredients if word]
l_ingredients

data[l_ingredients] = 0

In [6]:
data['ingredients_list'] = data['Cleaned_Ingredients']
data['ingredients_list'] = data.ingredients_list.apply(text_to_list)
data['ingredients_list'] = data.ingredients_list.apply(pun_list)
data['ingredients_list'] = data.ingredients_list.apply(num_list)
data['ingredients_list'] = data.ingredients_list.apply(low_list)
data['ingredients_list'] = data.ingredients_list.apply(sto_list)
data['ingredients_list'] = data.ingredients_list.apply(lem_list)

#Second iteration 

data['ingredients_list'] = data.ingredients_list.apply(sto_list)
data['ingredients_list'] = data.ingredients_list.apply(lem_list)
data['ingredients_list'] = data['ingredients_list'].apply(lambda x: [word for word in x if word])

In [7]:
#building ingredients table

all_ingredients =[]

for i in range (len(data['ingredients_list'])):
    for j in range (len(data['ingredients_list'][i])):
        all_ingredients.append(data['ingredients_list'][i][j])

all_ingredients_sentence = ' '.join([i for i in all_ingredients])       
list_ingredients = all_ingredients_sentence.split(" ")

df = pd.DataFrame(data = list_ingredients, columns =['word'])
df['count'] = 1
ingredients_df = pd.DataFrame(df.groupby('word')['count'].sum())

In [8]:
ingredients_df = ingredients_df.sort_values(by='count', ascending=False)

In [9]:
ingredients_df

,count
word,
oil,9171
juice,4516
lemon,4331
clove,4137
egg,4122
...,...
elderberry,1
einkorn,1
prebaked,1


In [10]:
data['ingredients_list']

0        [chicken, acorn squash total, sage, rosemary, ...
1              [egg, new potato diameter, rosemary, thyme]
2        [evaporated milk, milk, powder, onion powder, ...
3        [round italian loaf cube, oil, sweet italian s...
4        [dark, hot, bourbon, lemon juice, apple storeb...
                               ...                        
13496    [unsweetened cocoa powder, double acting powde...
13497    [preserved lemon, butternut squash seeded dice...
13498    [leftover katsuo bushi bonito flake making das...
13499    [baby spinach, feta crumbled scant, grated nut...
13500    [poblano chile, tomato including juice, clove,...
Name: ingredients_list, Length: 13501, dtype: object

In [11]:
#from pathlib import Path  
#filepath = Path('raw_data/ingredients_zach.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
#ingredients_df.to_csv(filepath)  

' '.join(data['ingredients_list'][9])

'basic lentil soup like amys onion turmeric cumin aleppo flake tomato paste egg ghee oil cumin oil finishing'

In [12]:
for columns_name in l_ingredients:
    for i in range(len(data['ingredients_list'])):
        ingredients = ' '.join(data['ingredients_list'][i])
        if columns_name in ingredients:
            data[columns_name][i] = 1       
        else:
            continue


/var/folders/vp/c987_zkx4hg6k0yfsdxsvmd80000gn/T/ipykernel_64028/1777820732.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_name][i] = 1


In [31]:
data['ingredients_list'][0]


KeyError: 'ingredients'